In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os, scipy
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as pyplot
# R interface
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
import rpy2.rinterface_lib.callbacks
import anndata2ri
import rpy2.robjects.numpy2ri
import loompy as lp
#import numpy2ri

pandas2ri.activate()
anndata2ri.activate()
rpy2.robjects.numpy2ri.activate()

%load_ext rpy2.ipython

In [ ]:
adata = sc.read('Data/adata_d50_d70.h5ad')

In [ ]:
%%R -i adata

Csparse_validate = "CsparseMatrix_validate"
library(Seurat)
library(edgeR)
library(tidyverse)

seurobj <- as.Seurat(adata, counts = "counts", data = NULL)

#seur <- readRDS("Data/d50_d70_neurons_seurat.rds")
seurobj <- RenameAssays(seurobj, originalexp="RNA")

seurobj[[]] %>% 
  group_by(diff_batch, dimensionality,Cell_types_2) %>% 
  dplyr::count() %>% 
  group_by(diff_batch, dimensionality) %>% 
  mutate(tot = sum(n), pct = n/tot) %>% 
  ggplot() +
  aes(pct, dimensionality, color=dimensionality) +
  geom_line(aes(group = diff_batch), color="grey80") +
  geom_point() +
  scale_x_continuous(labels=scales::percent) +
  facet_wrap(~(Cell_types_2), scales="free_y", ncol=1, strip.position = "left") +
  theme_bw() +
  theme(strip.background = element_blank(), legend.position = "top",  axis.title.y = element_blank(),
    axis.text.y = element_blank(),
    axis.ticks.y = element_blank(),
    axis.line.y = element_blank(),
    strip.text.y.left = element_text(angle = 0)) +
  labs(y=NULL, x=NULL, color=NULL)

In [ ]:
%%R
seur <- SplitObject(seurobj, split.by = "Cell_types_2")
table(seur[[1]]$reactionID,seur[[1]]$condition,seur[[1]]$diff_batch)
pb_list <- lapply(seur, function(x) {
  x[["group"]] <- interaction(x$reactionID,x$sampleID)
  y <- Seurat2PB(x, sample = "group", cluster = "condition")
  keep.samples <- y$samples$lib.size > 5e4
  keep.genes <- filterByExpr(y, group=y$samples$cluster)
  y <- y[keep.genes, , keep=FALSE]
  y <- normLibSizes(y)
  y$samples$rep <- gsub("[.].*","",y$samples$sample)
  y$samples$batch <- gsub("-.*","",gsub(".*[.]","",y$samples$sample))
  y$samples$day <- sub(".*-(.*)-.*", "\\1", y$samples$sample)
  return(y)
})

pb_list[[1]]$samples

In [ ]:
%%R
design_list <- lapply(pb_list, function(x) {
  condition <- relevel(factor(x$samples$cluster), ref="control")
  donor <- factor(x$samples$batch)
  day <- factor(x$samples$day)
  design <- model.matrix(~ condition+donor)
  colnames(design) <- gsub("donor", "", colnames(design))
  colnames(design)[1] <- "control"
  return(design)
})

In [ ]:
%%R
library(ggplot2)
#library(tidyverse)
head(colnames(design))
out <- voomLmFit(pb_list[[5]]$counts, design = design_list[[5]], block = pb_list[[5]]$samples$rep)
fit<- eBayes(out)

vplot <- topTable(fit,coef=3, n=Inf,adjust="BH", sort.by = "logFC") %>% rownames_to_column("gene") %>% 
  ggplot() + aes(logFC,-log10(P.Value)) + geom_point(aes(color=I(ifelse(adj.P.Val<0.05, "red","grey80")))) +
  ggrepel::geom_text_repel(aes(label = ifelse(abs(logFC)>1&adj.P.Val<0.05, gene, NA_character_))) +
  ggthemes::theme_clean()+ theme(plot.background = element_blank())

res <- purrr::map2_dfr(pb_list, design_list, function(x,y) {
  out <- voomLmFit(x$counts, design = y, block = x$samples$rep)
  fit<- eBayes(out)
  topTable(fit,coef=3, n=Inf,adjust="BH", sort.by = "logFC") %>% rownames_to_column("gene") %>% filter(gene %in% c("EGR1","FOS"))
}, .id="idx")

In [ ]:
%%R
lfcplot <- res %>% 
  mutate(idx = factor(idx)) %>% 
  ggplot() +
  aes(x=logFC, y=fct_rev(idx), color=I(ifelse(adj.P.Val<0.05, "red","grey80"))) +
  geom_point() +
  geom_vline(xintercept = 0, lty=2) +
  facet_wrap(~gene) +
  ggthemes::theme_few() +
  geom_rect(aes(xmin = -Inf, 
                xmax = Inf,
                ymax = 2.7,
                ymin = 3.3), fill = "red", alpha=1e-2)+
  scale_x_continuous(breaks = seq(-2, 2, by = 1)) +
  labs(y=NULL)

library(patchwork)
vplot+lfcplot+plot_layout(widths=c(2,1))
ggsave("tany.pdf", h=4, w=8, dpi = 320)